# Assignment 3

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model](https://open.spotify.com/track/4R3AU2pjv8ge2siX1fVbZs?si=b2712f32da0e4358) or a simple [linear regression model](https://scikit-learn.org/stable/modules/linear_model.html)
        - An advanced regressor of your choice (e.g., Random Forest, Neural Network, etc.)

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../data/adult/`, then you can use the code below to load them.

In [43]:
import pandas as pd
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = None, names = columns))
fires_dt

,coord_x,coord_y,month,day,ffmc,dmc,dc,isi,temp,rh,wind,rain,area
0,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
1,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0,0
2,7,4,oct,tue,90.6,35.4,669.1,6.7,18,33,0.9,0,0
3,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0,0
4,8,6,mar,fri,91.7,33.3,77.5,9,8.3,97,4,0.2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
513,4,3,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0,6.44
514,2,4,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0,54.29
515,7,4,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0,11.16
516,1,4,aug,sat,94.4,146,614.7,11.3,25.6,42,4,0,0


# Get X and Y

Create the features data frame and target data.

In [44]:
fires_dt['area'] = pd.to_numeric(fires_dt['area'], errors='coerce')
fires_dt.dropna(subset=['area'], inplace = True)

X = fires_dt.drop(columns=["area"], axis=1)
Y = fires_dt["area"]


# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [45]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler, StandardScaler, OneHotEncoder
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn import set_config

numerical_features = ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']
categorical_features =['month', 'day']

pipe_num_cols = Pipeline([
    ('standardizer', RobustScaler())
])

pipe_cat_cols = Pipeline([
   ('standardizer', OneHotEncoder(handle_unknown='infrequent_if_exist'))
])

preproc1 = ColumnTransformer([
    ('num', pipe_num_cols, numerical_features),
    ('cat', pipe_cat_cols, categorical_features)]
    , remainder='drop'
)


preproc1


ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('standardizer',
                                                  RobustScaler())]),
                                 ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                  'isi', 'temp', 'rh', 'wind', 'rain']),
                                ('cat',
                                 Pipeline(steps=[('standardizer',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist'))]),
                                 ['month', 'day'])])

### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [46]:
from sklearn.preprocessing import PowerTransformer

preproc2 = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('yeo_johnson', PowerTransformer(method='yeo-johnson')),
            ('scaler', StandardScaler())
        ]), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='infrequent_if_exist'), categorical_features)
    ]
)

preproc2

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('yeo_johnson',
                                                  PowerTransformer()),
                                                 ('scaler', StandardScaler())]),
                                 ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                  'isi', 'temp', 'rh', 'wind', 'rain']),
                                ('cat',
                                 OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                 ['month', 'day'])])

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [47]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

baseline_regressor = LinearRegression()
advanced_regressor = RandomForestRegressor(random_state=42)

pipeline_A = Pipeline(steps=[
    ('preprocessing', preproc1),
    ('regressor', baseline_regressor)
])

In [48]:
# Pipeline B = preproc2 + baseline
pipeline_B = Pipeline(steps=[
    ('preprocessing', preproc2),
    ('regressor', baseline_regressor)
])

In [49]:
# Pipeline C = preproc1 + advanced model
pipeline_C = Pipeline(steps=[
    ('preprocessing', preproc1),
    ('regressor', advanced_regressor)
])




In [50]:
# Pipeline D = preproc2 + advanced model

pipeline_D = Pipeline(steps=[
    ('preprocessing', preproc2),
    ('regressor', advanced_regressor)
])

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [51]:
from sklearn.model_selection import GridSearchCV

# Define parameter grids
param_grid_lr = {'regressor__fit_intercept': [True, False]}
param_grid_rf = {
    'regressor__n_estimators': [50, 100, 200],
    'regressor__max_features': ['sqrt', 'log2']
}

In [52]:
from sklearn.model_selection import GridSearchCV

grid_a = GridSearchCV(pipeline_A, param_grid_lr, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
grid_b = GridSearchCV(pipeline_B, param_grid_lr, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
grid_c = GridSearchCV(pipeline_C, param_grid_rf, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
grid_d = GridSearchCV(pipeline_D, param_grid_rf, cv=5, scoring='neg_mean_squared_error', return_train_score=True)


In [53]:
grid_a.fit(X, Y)
grid_b.fit(X, Y)
grid_c.fit(X, Y)
grid_d.fit(X, Y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('yeo_johnson',
                                                                                          PowerTransformer()),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['coord_x',
                                                                          'coord_y',
                                                                          'ffmc',
                                                                          'dmc',
                                                                          'dc',
                                                                          'isi',
                                                                          'temp',
                                                                          'rh',
                                                                          'wind',
                                                                          'rain']),
                                                                        ('cat',
                                                                         OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                                         ['month',
                                                                          'day'])])),
                                       ('regressor',
                                        RandomForestRegressor(random_state=42))]),
             param_grid={'regressor__max_features': ['sqrt', 'log2'],
                         'regressor__n_estimators': [50, 100, 200]},
             return_train_score=True, scoring='neg_mean_squared_error')

In [54]:
import joblib

best_model = grid_c.best_estimator_

def print_results(grid, name):
    print(f"Results for {name}")
    if hasattr(grid, 'best_params_') and hasattr(grid, 'best_score_'):
        print(f"Best Parameters: {grid.best_params_}")
        print(f"Best Score: {grid.best_score_}")
    else:
        print("The GridSearchCV object does not have the best_params_ or best_score_ attributes.")
    print("\n")

results = {
    'Pipeline A': grid_a.best_score_,
    'Pipeline B': grid_b.best_score_,
    'Pipeline C': grid_c.best_score_,
    'Pipeline D': grid_d.best_score_
}

best_model_name = min(results, key=results.get)
best_score = results[best_model_name]

print(f"Best model: {best_model_name} with a score of {best_score}")

# Save the best performing model to a pickle file
joblib.dump(best_model, 'best_model.pkl')

best_model_name = min(results, key=results.get)
best_score = results[best_model_name]

print(f"Best model: {best_model_name} with a score of {best_score}")

# Store the best estimator from the best model
if best_model_name == 'Pipeline A':
    best_model = grid_a.best_estimator_
elif best_model_name == 'Pipeline B':
    best_model = grid_b.best_estimator_
elif best_model_name == 'Pipeline C':
    best_model = grid_c.best_estimator_
else:
    best_model = grid_d.best_estimator_



Results for Pipeline A
Best Parameters: {'regressor__fit_intercept': True}
Best Score: -4429.800262484062


Results for Pipeline B
Best Parameters: {'regressor__fit_intercept': True}
Best Score: -4464.789032111983


Results for Pipeline C
Best Parameters: {'regressor__max_features': 'log2', 'regressor__n_estimators': 50}
Best Score: -4525.364430764396


Results for Pipeline D
Best Parameters: {'regressor__max_features': 'log2', 'regressor__n_estimators': 50}
Best Score: -4497.582074930576


Best model: Pipeline C with a score of -4525.364430764396
Best model: Pipeline C with a score of -4525.364430764396


# Evaluate

+ Which model has the best performance?

# Export

+ Save the best performing model to a pickle file.

# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

In [61]:
import pandas as pd
import shap
import matplotlib.pyplot as plt
import joblib

# Load the best model
best_model = joblib.load('best_model.pkl')

explainer = shap.Explainer(best_model.named_steps['regressor'], X)
shap_values = explainer(X)

# Plot SHAP values for a single prediction
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values[0,:], X.iloc[0,:])

# Save the plot as a file
plt.savefig('/mnt/data/shap_single_prediction.png')

# Plot SHAP summary plot for all features
shap.summary_plot(shap_values, X)

# Save the summary plot as a file
plt.savefig('/mnt/data/shap_summary_plot.png')

# Display paths of the saved files
shap_files = {
    "Single Prediction SHAP Plot": "/mnt/data/shap_single_prediction.png",
    "Summary SHAP Plot": "/mnt/data/shap_summary_plot.png"
}

shap_files


Found a NULL input array in _cext_dense_tree_update_weights!


TypeError: Cannot cast array data from dtype('O') to dtype('float64') according to the rule 'safe'

In [ ]:
shap.summary_plot(shap_values, X)

*(Answer here.)*

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-3`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_3.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.